In [87]:
from qtyaccounting import qtytools as qts

In [88]:
filename = "example/商品の仕入と売上.txt"
with open(filename, 'r',encoding='utf-8') as f:
    journal = f.read()

In [89]:
print(journal)

商品の仕入と売上の例

<<2022-01-01 &KIND::OPENING ##開始仕訳
Dr　預金 93,000円
Dr　商品#Tシャツ *10個 7,000円
Cr　資本金 100,000>>

<<2022-05-14 ##商品の仕入１
Dr　商品#Tシャツ *10個 6000円
Cr　預金 6000>>

<<2022-08-15 ##商品の仕入２
Dr　商品 #Tシャツ *10個 7000円 
Cr　預金 7000>>

<<2022-10-16 ##商品の売上
Dr 売上原価　#Tシャツ　?E円  $得意先A
Cr 商品 #Tシャツ　*5個 ? ##払出価額の自動計算>>

<<2022-12-16 $得意先A
Dr 売掛金 ?E円
Cr 売上高　#Tシャツ @200 *5個 &天気::晴れ &気温:26度　##売上計上>>


In [90]:
journal_tree = qts.QTYJournalToTree().translate(journal)
ledgers =qts.InterpretJournalTree().get_ledgers(journal_tree)

In [91]:
df = ledgers.simple_ledger_to_df("商品","","Tシャツ")
df

,entry_id,datetime,quantity,quantity_unit,amount,amount_unit,dr_quantity,dr_amount,cr_quantity,cr_amount,remarks
0,0,2022-01-01,10,個,7000,円,10.0,7000.0,NaN,NaN,開始仕訳開始仕訳
1,1,2022-05-14,10,個,6000,円,10.0,6000.0,NaN,NaN,商品の仕入１商品の仕入１
2,2,2022-08-15,10,個,7000,円,10.0,7000.0,NaN,NaN,商品の仕入２商品の仕入２
3,3,2022-10-16,5,個,OP_AUTO_AMOUNT,NaN,NaN,NaN,5.0,3500.0,払出価額の自動計算商品の売上


In [92]:
ledgers.simple_ledger_to_excel("商品","","Tシャツ")

In [93]:
df_tb = ledgers.tb_to_df(start_datetime="2022-05-01",end_datetime="2023-01-01")
df_tb

,account,sub_account,item,start_quantity,start_amount,sum_dr_quantity,sum_dr_amount,sum_cr_quantity,sum_cr_amount,end_quantity,end_amount
0,預金,,,1,93000,0,0.0,2,13000.0,-1,80000.0
1,売掛金,,,0,0,1,1000.0,0,0.0,1,1000.0
2,商品,,Tシャツ,10,7000,20,13000.0,5,3500.0,25,16500.0
3,資本金,,,1,100000,0,0.0,0,0.0,1,100000.0
4,売上高,,Tシャツ,0,0,0,0.0,5,1000.0,5,1000.0
5,売上原価,,Tシャツ,0,0,1,3500.0,0,0.0,1,3500.0


In [94]:
ledgers.tb_to_excel(start_datetime="2022-05-01",end_datetime="2023-01-01")

In [95]:
lgs_df =ledgers.get_record_df()
lgs_df

,entry_id,counts_dr_cr,order_id,line_no,datetime,account,sub_account,item,partner,person_in_charge,...,amount,amount_unit,dr_quantity,dr_amount,cr_quantity,cr_amount,remarks,KIND,天気,気温
0,0,3,0,0,2022-01-01,預金,,,NaN,NaN,...,93000,円,1.0,93000.0,NaN,NaN,開始仕訳開始仕訳,OPENING,NaN,NaN
1,0,3,2,0,2022-01-01,資本金,,,NaN,NaN,...,100000,NaN,NaN,NaN,1.0,100000.0,開始仕訳開始仕訳,OPENING,NaN,NaN
2,0,3,1,1,2022-01-01,商品,,Tシャツ,NaN,NaN,...,7000,円,10.0,7000.0,NaN,NaN,開始仕訳開始仕訳,OPENING,NaN,NaN
3,1,2,0,0,2022-05-14,商品,,Tシャツ,NaN,NaN,...,6000,円,10.0,6000.0,NaN,NaN,商品の仕入１商品の仕入１,NaN,NaN,NaN
4,1,2,1,0,2022-05-14,預金,,,NaN,NaN,...,6000,NaN,NaN,NaN,1.0,6000.0,商品の仕入１商品の仕入１,NaN,NaN,NaN
5,2,2,0,0,2022-08-15,商品,,Tシャツ,NaN,NaN,...,7000,円,10.0,7000.0,NaN,NaN,商品の仕入２商品の仕入２,NaN,NaN,NaN
6,2,2,1,0,2022-08-15,預金,,,NaN,NaN,...,7000,NaN,NaN,NaN,1.0,7000.0,商品の仕入２商品の仕入２,NaN,NaN,NaN
7,3,2,0,0,2022-10-16,売上原価,,Tシャツ,得意先A,NaN,...,OP_EQUAL_AMOUNT,円,1.0,3500.0,NaN,NaN,商品の売上商品の売上,NaN,NaN,NaN
8,3,2,1,0,2022-10-16,商品,,Tシャツ,NaN,NaN,...,OP_AUTO_AMOUNT,NaN,NaN,NaN,5.0,3500.0,払出価額の自動計算商品の売上,NaN,NaN,NaN
9,4,2,0,0,2022-12-16,売掛金,,,得意先A,NaN,...,OP_EQUAL_AMOUNT,円,1.0,1000.0,NaN,NaN,,NaN,NaN,NaN


In [96]:
# 例：天気や気温と売上数量、売上高との関係
lgs_df[lgs_df["account"]=="売上高"][["天気","気温","cr_quantity","cr_amount"]]

,天気,気温,cr_quantity,cr_amount
10,晴れ,26.0,5.0,1000.0
